In [14]:
# Import necessary libraries
import warnings
warnings.filterwarnings('ignore')
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
plt.style.use('ggplot')

In [15]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [16]:
from keras.models import Sequential
from keras.layers import Activation, Dense, LSTM, Dropout

In [17]:
# Load the dataset
dataset = sm.datasets.co2.load_pandas().data

# Convert into DataFrame
df = pd.DataFrame(dataset)

# Update to datetime type
pd.to_datetime(df.index)

# The term bfill means that we use the value before filling in missing values
df.bfill(inplace = True)

In [18]:
# Train/Test Split
train = df[:'1990-01']
test = df['1990-01':]

In [19]:
df.head()

,co2
1958-03-29,316.1
1958-04-05,317.3
1958-04-12,317.6
1958-04-19,317.5
1958-04-26,316.4


In [20]:
len(train)

1662

In [21]:
len(test)

626

## Vanilla LSTM 

In [26]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [34]:
# Setting raw sequence to all data points
raw_seq = train.values

# Determining number of steps after which to split sequence
n_steps = 14

In [35]:
# Splitting the sequence
X, y = split_sequence(raw_seq, n_steps)

In [36]:
# Number of features is one because time series is univariate
n_features = 1

In [37]:
# Reshape from (samples, timesteps) into (samples, timesteps, features)
X = X.reshape(X.shape[0], X.shape[1], n_features)

In [38]:
# Define the model
model = Sequential()
model.add(LSTM(50, activation = 'relu', input_shape = (n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse')

In [39]:
# Fit the model
model.fit(X, y, epochs = 200, verbose = 0)

In [45]:
# Predict on unseen values

# Define our prediction x values
x_predict = test[:14].values

# Reshape the prediction vector into a matrix
x_predict = x_predict.reshape(1, n_steps, n_features)

In [46]:
# Predict 
y_hat = model.predict(x_predict)

In [47]:
y_hat

array([[355.31946]], dtype=float32)

In [54]:
test.values[15]

array([356.6])

## Stacked LSTM

LSTMs have a two dimensional output by default. In order to stack LSTM layers, set 'return_sequences' to true, which will produce a three dimensional output from the first layer and can be used as an input for the second layer.

In [62]:
# Define the model
model = Sequential()
model.add(LSTM(50, activation = 'relu', return_sequences = True, input_shape = (n_steps, n_features)))
model.add(LSTM(50, activation = 'relu'))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse')

In [63]:
# Fit the model
model.fit(X, y, epochs = 100, verbose = 0)

In [64]:
# Predict on unseen values

# Define our prediction x values
x_predict = test[:14].values

# Reshape the prediction vector into a matrix
x_predict = x_predict.reshape(1, n_steps, n_features)

In [65]:
# Predict 
y_hat = model.predict(x_predict)

In [66]:
y_hat

array([[352.4363]], dtype=float32)

In [67]:
test.values[15]

array([356.6])

## Bidirectional LSTM

Sometimes the model will do better if it can read the sequence both forward and backward. Wrap the LSTM layer in a 'Bidirectional' layer.

In [71]:
from keras.layers import Bidirectional

In [72]:
# Define the model
model = Sequential()
model.add(Bidirectional(LSTM(50, activation = 'relu', input_shape = (n_steps, n_features))))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse')

In [73]:
# Fit the model
model.fit(X, y, epochs = 100, verbose = 0)

In [74]:
# Predict on unseen values

# Define our prediction x values
x_predict = test[:14].values

# Reshape the prediction vector into a matrix
x_predict = x_predict.reshape(1, n_steps, n_features)

In [75]:
# Predict 
y_hat = model.predict(x_predict)

In [76]:
y_hat

array([[356.89163]], dtype=float32)

In [77]:
test.values[15]

array([356.6])

## CNN LSTM

no idea how a CNN works

In [78]:
from keras.layers import TimeDistributed
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

## ConvLSTM 

## Multiple Input Series

In [79]:
df1 = df

In [81]:
df1['root'] = [np.sqrt(x) for x in df1['co2']]

In [84]:
df1.head(1)

,co2,root
1958-03-29,316.1,17.779201


In [85]:
# Train/Test Split
train1 = df1[:'1990-01']
test1 = df1['1990-01':]

Express output as a sum of the two input rows

In [88]:
out_seq = train1['co2'].values + train1['root'].values

Reshape inputs and output into a dataframe where each row represents a timestep and each column is a separate time series.

In [92]:
# Convert to (rows, columns) structure
inseq1 = train1['co2'].values.reshape((len(train1['co2']), 1))
inseq2 = train1['root'].values.reshape((len(train1['root']), 1))
outseq = out_seq.reshape((len(out_seq), 1))
# Horizontally stack arrays into a dataframe
multi = np.hstack((inseq1, inseq2, outseq))

In [96]:
multi

array([[316.1       ,  17.77920133, 333.87920133],
       [317.3       ,  17.81291666, 335.11291666],
       [317.6       ,  17.82133553, 335.42133553],
       ...,
       [353.5       ,  18.80159568, 372.30159568],
       [353.8       ,  18.80957203, 372.60957203],
       [353.9       ,  18.81223006, 372.71223006]])

As with univariate time series, the data must be split into samples with inputs and outputs

In [97]:
# split a multivariate sequence into samples
def split_multi_sequences(sequences, n_steps):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the dataset
        if end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)